<a href="https://colab.research.google.com/github/hridog00/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Helena_Ridocci_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad Guiada 3 - AG3 <br>
Helena Ridocci <br>
URL: https://colab.research.google.com/drive/17EkhzI3aWrAMJboz3GAua69-x9Bkbp2E

In [0]:
!pip install requests
!pip install tsplib95

In [0]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7fa1ab3686a0>)

In [0]:
import random
import tsplib95
import math 

In [0]:
problem = tsplib95.load_problem(file)

In [0]:
nodos = list(problem.get_nodes())
edges = list(problem.get_edges())

In [0]:
print('Numero de ciudades :', len(nodos))

Numero de ciudades : 42


In [0]:
type(nodos)

list

In [0]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)

#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [0]:
solucion = crear_solucion(nodos)
distancia_total(solucion,problem)

5164

Busqueda aleatoria

In [0]:
def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion(nodos)
  mejor_valor    = distancia_total(mejor_solucion, problem)

  for i in range(N):
      solucion = crear_solucion(nodos)
      valor    = distancia_total(solucion, problem)
      if valor < mejor_valor:
        mejor_solucion = solucion
        mejor_valor = valor

  print("La mejor solucion es ",mejor_valor,  mejor_solucion)
   
busqueda_aleatoria(problem, 1000)

La mejor solucion es  3719 [0, 2, 1, 4, 10, 20, 33, 17, 31, 22, 12, 27, 19, 30, 38, 39, 21, 23, 28, 32, 34, 35, 16, 18, 40, 8, 29, 5, 14, 41, 11, 24, 9, 3, 7, 13, 15, 25, 37, 36, 6, 26]


Busqueda local


In [0]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  #print(mejor_distancia, mejor_solucion)
  return mejor_solucion
      
solucion = crear_solucion(nodos)
mejor_solucion = genera_vecina(solucion)


In [0]:
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
    mejor_solucion = []
    mejor_distancia = 10e100
  
    solucion_referencia = crear_solucion(nodos)
  
    for i in range(N):
        vecina = genera_vecina(solucion_referencia)
        distancia_vecina = distancia_total(vecina, problem)
        if distancia_vecina <= mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
        
        solucion_referencia = vecina
    print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)
    return mejor_solucion
mejor_solucion = busqueda_local(problem, 100)

La mejor solución encontrada es  1615 [0, 31, 17, 36, 35, 20, 33, 34, 32, 18, 12, 11, 10, 8, 29, 2, 27, 1, 14, 16, 15, 37, 7, 28, 30, 38, 22, 39, 24, 40, 21, 9, 23, 41, 25, 13, 19, 5, 26, 6, 4, 3]


Rcorrido simulado

In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False


def bajar_temperatura(T):
  return T*.9999

#####################################################################
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100

  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=100000)

	****Encontrada mejor solucion: 4970 [0, 33, 34, 13, 27, 12, 40, 20, 4, 8, 41, 19, 2, 31, 5, 29, 22, 37, 23, 28, 7, 16, 18, 38, 14, 15, 35, 25, 17, 21, 10, 3, 24, 32, 6, 9, 39, 1, 11, 26, 30, 36] 

	****Encontrada mejor solucion: 4872 [0, 33, 34, 35, 27, 12, 40, 20, 9, 8, 41, 19, 2, 31, 5, 29, 22, 37, 23, 28, 7, 16, 18, 38, 14, 15, 4, 25, 17, 21, 10, 3, 24, 32, 6, 13, 39, 1, 11, 26, 30, 36] 

	****Encontrada mejor solucion: 4824 [0, 33, 34, 35, 27, 12, 40, 20, 9, 8, 41, 19, 2, 31, 5, 29, 22, 37, 23, 36, 7, 16, 18, 38, 14, 15, 4, 25, 17, 21, 10, 3, 24, 32, 6, 13, 39, 1, 11, 26, 30, 28] 

	****Encontrada mejor solucion: 4763 [0, 33, 34, 35, 27, 12, 40, 20, 9, 8, 41, 19, 2, 31, 5, 29, 22, 37, 23, 36, 7, 16, 18, 38, 14, 11, 4, 25, 15, 21, 10, 3, 24, 32, 6, 13, 26, 1, 17, 39, 30, 28] 

	****Encontrada mejor solucion: 4651 [0, 33, 34, 35, 27, 12, 40, 20, 23, 8, 29, 19, 2, 31, 38, 41, 22, 37, 9, 36, 16, 7, 18, 5, 14, 11, 4, 25, 15, 21, 10, 6, 24, 32, 3, 13, 26, 1, 17, 28, 30, 39] 

	****Encon

Colonia de hormigas

In [0]:
def get_prob(nodos, T,i, j, nodos_sin_visitar, alpha = 0.1, beta=0.5):
    if j not in nodos_sin_visitar:
        return 0
    p = ((T[i][j])**alpha * (1/distancia(i, j, problem))**beta) / sum([ (T[i][l])**alpha * (1/distancia(i, l, problem))**beta for l in nodos_sin_visitar ])
    return p

In [0]:
from numpy.random import choice
def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  nodos = list(problem.get_nodes())
  nodos_sin_visitar = list(set(range(1,len(nodos))) - set(H) )  
  p = [get_prob(nodos, T, H[-1], nodo, nodos_sin_visitar) for nodo in nodos]

  #return random.choice(   list(set(range(1,len(nodos))) - set(H) )  )
  return choice(nodos, p=p)


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(nodos)) ] for j in range(len(nodos))]
  return T 

In [0]:
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(nodos)) ] for _ in range(len(nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print('Mejor distancia:',mejor_distancia)
  return mejor_distancia

In [0]:
import numpy as np
np.mean([hormigas(problem, 1000) for _ in range(10)])

[0, 29, 37, 7, 17, 2, 28, 27, 1, 26, 15, 4, 32, 30, 23, 9, 10, 11, 25, 12, 21, 19, 34, 35, 36, 31, 14, 16, 6, 3, 18, 33, 39, 41, 24, 40, 22, 13, 5, 8, 20, 38]
Mejor distancia: 3381
[0, 20, 35, 8, 22, 41, 32, 27, 21, 40, 34, 39, 30, 25, 10, 3, 7, 6, 1, 4, 38, 23, 24, 9, 29, 28, 12, 11, 19, 13, 36, 31, 2, 33, 16, 15, 14, 5, 37, 18, 26, 17]
Mejor distancia: 3494
[0, 37, 16, 7, 10, 41, 8, 18, 32, 33, 23, 21, 28, 26, 6, 31, 13, 5, 4, 2, 3, 1, 40, 12, 36, 17, 20, 38, 29, 30, 39, 24, 9, 22, 25, 11, 34, 15, 35, 14, 19, 27]
Mejor distancia: 3487
[0, 4, 36, 37, 15, 14, 16, 5, 13, 18, 28, 29, 17, 34, 20, 39, 38, 19, 26, 32, 8, 21, 41, 24, 22, 23, 30, 10, 25, 3, 27, 2, 35, 33, 31, 40, 9, 7, 1, 6, 12, 11]
Mejor distancia: 3295
[0, 17, 37, 16, 32, 20, 15, 14, 19, 13, 1, 27, 4, 35, 7, 6, 3, 9, 12, 5, 29, 28, 8, 11, 39, 18, 10, 23, 41, 26, 21, 40, 30, 25, 22, 24, 33, 34, 31, 36, 2, 38]
Mejor distancia: 3374
[0, 32, 1, 34, 6, 26, 3, 2, 36, 5, 7, 22, 41, 9, 19, 18, 10, 11, 12, 17, 37, 15, 14, 13, 8, 23,

3352.0